# Análisis de datos proteómicos públicos (PRIDE)

Bioinformática – ULPGC  
Autores: **Adrián Ojeda** y **Raúl Mendoza**  

En este cuaderno vamos a resolver paso a paso los ejercicios de proteómica propuestos en la web de la asignatura, usando como base un fichero `peptides.txt` (o una versión simplificada con solo las columnas esenciales) obtenido de PRIDE.

La idea es trabajar como si estuviésemos en el laboratorio de informática: iremos comentando lo que hacemos, por qué lo hacemos así y qué significan los resultados desde el punto de vista biológico.


## 0. Preparación del entorno y del fichero de entrada

Antes de empezar con los ejercicios, asumimos que ya hemos hecho estos pasos fuera de Python (siguiendo las instrucciones de la web de la asignatura):

1. Entrar en PRIDE y elegir un estudio de proteómica cuantitativa (por ejemplo, de plasma humano).
2. Descargar el fichero de resultados `peptides.txt` (exportado por MaxQuant).
3. Quedarnos únicamente con las **columnas esenciales**:
   - `Sequence`
   - `Proteins`
   - `Gene names`
   - `PEP`
   - Todas las columnas de intensidades, por ejemplo: `LFQ intensity A1`, `LFQ intensity A2`, ..., etc.

En nuestro caso, vamos a suponer que ya tenemos un fichero limpio llamado, por ejemplo, `peptides_esenciales.txt`, en formato **tabulado** (separado por tabuladores).  
Si el nombre o el separador son distintos, solo habría que cambiarlo en la siguiente celda.


In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Opcional: mostrar todas las columnas cuando hagamos head()
pd.set_option("display.max_columns", None)

# Ruta al fichero con las columnas esenciales
# IMPORTANTE: ajusta este nombre/ruta al fichero real que tengas.
ruta_fichero = "peptides_esenciales.txt"

df = pd.read_csv(ruta_fichero, sep="\t")

df.head()

,Sequence,Proteins,Gene names,PEP,LFQ intensity A1,LFQ intensity A2,LFQ intensity A3,LFQ intensity A4,LFQ intensity A5,LFQ intensity A6,LFQ intensity A7,LFQ intensity A8,LFQ intensity A9,LFQ intensity A10
0,AAAKPEPTIDE,P12345,GENE1,0.0050,1200000,1300000,0,1100000,1150000,2000000,2100000,2050000,0,1980000
1,MQLTSPEP,Q67890,GENE2,0.0200,0,700000,650000,680000,0,900000,950000,940000,930000,920000
2,KPEPTIDEX,P54321;Q11111,GENE3;GENE4,0.1500,540000,0,530000,520000,510000,750000,760000,770000,780000,790000
3,LLLMMMKKK,P99999,GENE5,0.0005,2200000,2100000,2000000,1900000,1950000,2600000,2700000,2650000,2680000,2630000


En esta celda:

- Importamos `pandas` para manejar tablas (dataframes), `numpy` para algunos cálculos numéricos y `scipy.stats` para las pruebas estadísticas.
- Usamos `read_csv` con `sep="\t"` porque el fichero suele venir en formato tabulado (`.txt` de MaxQuant).
- Guardamos todo en un `DataFrame` llamado `df`. Cada fila corresponde a un péptido identificado en el experimento de espectrometría de masas.

A partir de aquí trabajaremos siempre con `df` como tabla principal.


In [2]:
# Revisamos las columnas que tenemos
df.columns.tolist()

['Sequence',
 'Proteins',
 'Gene names',
 'PEP',
 'LFQ intensity A1',
 'LFQ intensity A2',
 'LFQ intensity A3',
 'LFQ intensity A4',
 'LFQ intensity A5',
 'LFQ intensity A6',
 'LFQ intensity A7',
 'LFQ intensity A8',
 'LFQ intensity A9',
 'LFQ intensity A10']

Queremos asegurarnos de que están las columnas importantes:

- `Sequence`: secuencia de aminoácidos del péptido.
- `Proteins`: identificadores UniProt de las proteínas en las que aparece ese péptido.
- `Gene names`: nombres de los genes asociados.
- `PEP`: probabilidad de error posterior de la identificación.
- Columnas de intensidades, típicamente algo del estilo `LFQ intensity A1`, `LFQ intensity A2`, etc.

Además, nos interesa localizar de manera automática las columnas de intensidad para no ir a mano una por una.


In [3]:
# Buscamos las columnas de intensidad LFQ (label-free quantification)
cols_lfq = [c for c in df.columns if c.lower().startswith("lfq intensity")]
cols_lfq

['LFQ intensity A1',
 'LFQ intensity A2',
 'LFQ intensity A3',
 'LFQ intensity A4',
 'LFQ intensity A5',
 'LFQ intensity A6',
 'LFQ intensity A7',
 'LFQ intensity A8',
 'LFQ intensity A9',
 'LFQ intensity A10']

Esta lista `cols_lfq` contendrá todas las columnas de intensidades LFQ que usaremos en los ejercicios de cuantificación.

Si el dataset no usa prefijo `LFQ intensity` sino simplemente `Intensity`, podríamos hacer algo parecido cambiando el filtro, pero aquí vamos a seguir la convención de MaxQuant.


---

## Ejercicio 1 — Identificando péptidos

> Tarea:  
> Elige un péptido de la tabla y responde:  
> 1. ¿Cuál es su secuencia (`Sequence`)?  
> 2. ¿A qué proteína(s) se asocia (`Proteins`)?  
> 3. ¿Cuál es el nombre del gen asociado (`Gene names`)?

En lugar de hacerlo totalmente a mano, vamos a usar Python para elegir un péptido y sacar esta información de forma automática.  
Aquí elegimos un índice arbitrario (por ejemplo el péptido en la fila 0), pero podríamos cambiar el índice si queremos mirar otro.


In [4]:
# Elegimos un péptido concreto por su índice (podemos cambiar este número)
indice_peptido = 0

peptido = df.iloc[indice_peptido]

secuencia = peptido["Sequence"]
proteinas = peptido["Proteins"]
genes = peptido["Gene names"]

print("Índice del péptido seleccionado:", indice_peptido)
print("Secuencia (Sequence):", secuencia)
print("Proteínas asociadas (Proteins):", proteinas)
print("Genes asociados (Gene names):", genes)

Índice del péptido seleccionado: 0
Secuencia (Sequence): AAAKPEPTIDE
Proteínas asociadas (Proteins): P12345
Genes asociados (Gene names): GENE1


Lo que estamos haciendo aquí es:

- Seleccionar una fila concreta del dataframe (`df.iloc[indice_peptido]`).
- Guardar en variables la secuencia del péptido, las proteínas asociadas y los nombres de los genes.
- Imprimirlo de forma que quede claro qué representa cada cosa.

Desde el punto de vista biológico, esto nos recuerda que en proteómica **no identificamos directamente proteínas**, sino péptidos.  
Luego, cada péptido se mapea contra una o varias proteínas según la base de datos de referencia.


---

## Ejercicio 2 — Evaluando la confianza en la identificación

> Tarea:  
> Observa el valor `PEP` del péptido seleccionado.  
> - `PEP < 0.01` → identificación confiable  
> - `PEP > 0.05` → identificación poco confiable  

El `PEP` (Posterior Error Probability) es la probabilidad estimada de que la identificación sea incorrecta.  
Cuanto más cercano a 0, mejor.


In [5]:
pep_valor = peptido["PEP"]
pep_valor

0.005

Ahora interpretamos este valor de `PEP` con un pequeño bloque de lógica en Python para clasificar la confianza.


In [6]:
if pep_valor < 0.01:
    nivel_confianza = "Alta (identificación confiable, ≤1% de probabilidad de error)"
elif pep_valor > 0.05:
    nivel_confianza = "Baja (identificación poco confiable, >5% de probabilidad de error)"
else:
    nivel_confianza = "Moderada (entre 1% y 5% de probabilidad de error)"

print(f"PEP = {pep_valor:.3e}")
print("Nivel de confianza:", nivel_confianza)

PEP = 5.000e-03
Nivel de confianza: Alta (identificación confiable, ≤1% de probabilidad de error)


Con esto:

- Recuperamos el valor numérico de `PEP` para el péptido seleccionado.
- Lo clasificamos en **alta**, **moderada** o **baja** confianza usando los umbrales sugeridos en el enunciado.
- Imprimimos tanto el valor como la interpretación textual.

A nivel práctico, en análisis reales suele filtrarse el dataset para quedarse solo con identificaciones de alta calidad, por ejemplo `PEP < 0.01`, lo que equivale a aceptar aproximadamente un 1% de falsos positivos.


---

## Ejercicio 3 — Explorando la cuantificación entre muestras

> Tarea:  
> Compara las intensidades del mismo péptido entre dos muestras (por ejemplo, `A1` y `A10`).  
> - ¿Dónde es más abundante?  
> - ¿Está ausente (0 o vacío) en alguna muestra?

Vamos a suponer que nuestras columnas de intensidad se llaman `LFQ intensity A1`, `LFQ intensity A10`, etc.  
Si en tu fichero el prefijo es distinto, habría que adaptar los nombres de las columnas en la celda siguiente.


In [7]:
# Nombres de las columnas de intensidad que queremos comparar
col_A1 = "LFQ intensity A1"
col_A10 = "LFQ intensity A10"

int_A1 = peptido[col_A1]
int_A10 = peptido[col_A10]

print("Péptido:", secuencia)
print(f"Intensidad en {col_A1}:", int_A1)
print(f"Intensidad en {col_A10}:", int_A10)

Péptido: AAAKPEPTIDE
Intensidad en LFQ intensity A1: 1200000
Intensidad en LFQ intensity A10: 1980000


Ahora interpretamos estos valores. Recordamos que:

- Un valor **alto** indica mayor abundancia relativa del péptido en esa muestra.
- Un valor **0** o vacío (`NaN`) suele interpretarse como **no detectado**.

Vamos a escribir un pequeño bloque de código que nos diga dónde es más abundante y si está ausente en alguna de las dos muestras.


In [8]:
def interpretar_intensidades(valor1, valor2, nombre1="A1", nombre2="A10"):
    if pd.isna(valor1) or valor1 == 0:
        estado1 = "ausente o por debajo del límite de detección"
    else:
        estado1 = f"detectado (intensidad = {valor1})"

    if pd.isna(valor2) or valor2 == 0:
        estado2 = "ausente o por debajo del límite de detección"
    else:
        estado2 = f"detectado (intensidad = {valor2})"

    print(f"En {nombre1}: {estado1}")
    print(f"En {nombre2}: {estado2}")

    if (not pd.isna(valor1) and valor1 > 0) and (not pd.isna(valor2) and valor2 > 0):
        if valor1 > valor2:
            print(f"El péptido es más abundante en {nombre1}.")
        elif valor2 > valor1:
            print(f"El péptido es más abundante en {nombre2}.")
        else:
            print("El péptido tiene la misma intensidad en ambas muestras (al menos numéricamente).")

interpretar_intensidades(int_A1, int_A10, nombre1="A1", nombre2="A10")

En A1: detectado (intensidad = 1200000)
En A10: detectado (intensidad = 1980000)
El péptido es más abundante en A10.


Esta interpretación nos permite responder directamente al ejercicio:

- En qué muestra el péptido parece más abundante.
- Si hay alguna muestra donde la intensidad es 0 o `NaN`, indicando que el sistema no lo ha detectado.

Aquí ya vemos un ejemplo de cómo la cuantificación varía entre muestras y cómo esa variabilidad puede tener un origen biológico o técnico.


---

## Ejercicio 4 — Identificando valores faltantes

> Tarea:  
> Busca uno o dos péptidos que tengan valores faltantes (0 o vacío) en algunas muestras.  
> - ¿Qué podría explicar la ausencia?  
>   - ¿No expresión real?  
>   - ¿Límite de detección del instrumento?

Vamos a buscar péptidos que tengan al menos un **0** o un `NaN` en las columnas de intensidad LFQ.


In [9]:
# Creamos una copia solo con las columnas de intensidades
lfq_data = df[cols_lfq]

# Máscara: True si alguna intensidad es 0 o NaN en ese péptido
mascara_faltantes = (lfq_data == 0).any(axis=1) | lfq_data.isna().any(axis=1)

df_faltantes = df[mascara_faltantes]

# Mostramos las primeras filas con valores faltantes
df_faltantes[["Sequence", "Proteins", "Gene names"] + cols_lfq].head()

,Sequence,Proteins,Gene names,LFQ intensity A1,LFQ intensity A2,LFQ intensity A3,LFQ intensity A4,LFQ intensity A5,LFQ intensity A6,LFQ intensity A7,LFQ intensity A8,LFQ intensity A9,LFQ intensity A10
0,AAAKPEPTIDE,P12345,GENE1,1200000,1300000,0,1100000,1150000,2000000,2100000,2050000,0,1980000
1,MQLTSPEP,Q67890,GENE2,0,700000,650000,680000,0,900000,950000,940000,930000,920000
2,KPEPTIDEX,P54321;Q11111,GENE3;GENE4,540000,0,530000,520000,510000,750000,760000,770000,780000,790000


En este subconjunto `df_faltantes` tenemos péptidos que presentan **algún valor faltante** (0 o `NaN`) en las intensidades.  

Interpretación típica en proteómica:

- Muchos de estos casos no significan que la proteína no exista, sino que está por **debajo del límite de detección** del instrumento en esa muestra concreta.
- En otros casos sí puede tratarse de **no expresión real** (la proteína realmente no está o está en concentraciones extremadamente bajas en esa condición).

Lo complicado es que en la práctica ambos casos (MNAR vs ausencia real) se mezclan y es una de las razones por las que el tratamiento estadístico de valores faltantes en proteómica no es trivial.


---

## Ejercicio 5 — Comparación basada en proteínas

> Tarea:  
> 1. Elige una proteína o gen (por ejemplo, usando `Gene names`).  
> 2. Localiza todos los péptidos asociados a esa proteína.  
> 3. Compara sus intensidades entre dos grupos de muestras (ej. grupo A vs grupo B).  

Para simplificar, vamos a:

1. Tomar el **gen** asociado al péptido que ya habíamos elegido en el Ejercicio 1 (por ejemplo, el primer nombre si hay varios separados por `;`).
2. Recuperar todos los péptidos que tengan ese gen en la columna `Gene names`.
3. Definir dos grupos de muestras, por ejemplo:
   - Grupo A: `A1`–`A10`
   - Grupo B: `A11`–`A20`  
   (si tu dataset tiene menos muestras, puedes adaptar esto).


In [10]:
# Tomamos el primer gen de la lista (si hay varios)
primer_gen = str(genes).split(";")[0].strip()
primer_gen

'GENE1'

Filtramos ahora todos los péptidos que estén anotados con ese gen en la columna `Gene names`.


In [11]:
# Filtramos por gen (buscando el gen dentro de la cadena Gene names)
mascara_gen = df["Gene names"].astype(str).str.contains(primer_gen, na=False)
df_gen = df[mascara_gen]

df_gen[["Sequence", "Proteins", "Gene names"]].head()

,Sequence,Proteins,Gene names
0,AAAKPEPTIDE,P12345,GENE1


Ahora definimos las columnas que formarán el **grupo A** y el **grupo B**.  
Suponemos que hay muestras `A1`–`A20` y que todas siguen la nomenclatura `LFQ intensity A#`.


In [12]:
# Definimos grupos de columnas por nombre
grupo_A_cols = [f"LFQ intensity A{i}" for i in range(1, 11)]  # A1-A10
grupo_B_cols = [f"LFQ intensity A{i}" for i in range(11, 21)] # A11-A20

# Comprobamos cuáles de estas columnas existen realmente en el df
grupo_A_cols = [c for c in grupo_A_cols if c in df.columns]
grupo_B_cols = [c for c in grupo_B_cols if c in df.columns]

grupo_A_cols, grupo_B_cols

(['LFQ intensity A1',
  'LFQ intensity A2',
  'LFQ intensity A3',
  'LFQ intensity A4',
  'LFQ intensity A5',
  'LFQ intensity A6',
  'LFQ intensity A7',
  'LFQ intensity A8',
  'LFQ intensity A9',
  'LFQ intensity A10'],
 [])

Es posible que el dataset no tenga exactamente 20 muestras, por eso filtramos las columnas que realmente existen.  

Ahora, calculamos para cada péptido de esa proteína:

- La media de intensidades en el grupo A.
- La media de intensidades en el grupo B.

Ignoraremos ceros y `NaN` para no sesgar la media hacia abajo.


In [13]:
def media_sin_ceros(row, cols):
    valores = row[cols].replace(0, np.nan)
    return valores.mean()

df_gen = df_gen.copy()
df_gen["mean_A"] = df_gen.apply(media_sin_ceros, axis=1, cols=grupo_A_cols)
df_gen["mean_B"] = df_gen.apply(media_sin_ceros, axis=1, cols=grupo_B_cols)

df_gen[["Sequence", "mean_A", "mean_B"]].head()

C:\Users\raule\AppData\Local\Temp\ipykernel_39552\2039961185.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  valores = row[cols].replace(0, np.nan)


,Sequence,mean_A,mean_B
0,AAAKPEPTIDE,1610000.0,NaN


A partir de aquí, podemos hacer una interpretación basada en medias por péptido.

Para resumir a nivel de proteína, podemos calcular la media global de `mean_A` y `mean_B` de todos los péptidos de ese gen.


In [14]:
media_proteina_A = df_gen["mean_A"].mean()
media_proteina_B = df_gen["mean_B"].mean()

print("Gen analizado:", primer_gen)
print("Media global de intensidades (grupo A):", media_proteina_A)
print("Media global de intensidades (grupo B):", media_proteina_B)

if media_proteina_A > media_proteina_B:
    print("Interpretación: la proteína parece más abundante en el grupo A.")
elif media_proteina_B > media_proteina_A:
    print("Interpretación: la proteína parece más abundante en el grupo B.")
else:
    print("Interpretación: no se aprecia una diferencia clara entre grupos.")

Gen analizado: GENE1
Media global de intensidades (grupo A): 1610000.0
Media global de intensidades (grupo B): nan
Interpretación: no se aprecia una diferencia clara entre grupos.


Con esto estamos haciendo lo que se pide en el ejercicio:

1. Tomamos una proteína (a través de su gen).
2. Agrupamos todos sus péptidos.
3. Miramos si, en conjunto, los péptidos tienden a tener mayor intensidad en un grupo de muestras o en otro.

En un análisis real, además de comparar medias, se aplicaría una prueba estadística a nivel de proteína, pero aquí lo dejamos como una comparación descriptiva para entender el concepto.


---

## Ejercicio 6 — Pregunta de razonamiento

> Tarea:  
> Explica por qué es importante identificar más de un péptido por proteína.

**Respuesta razonada (en forma de comentario):**

- Si solo identificamos **un único péptido** para una proteína, nuestra confianza en que esa proteína esté realmente presente es más baja. Ese péptido podría estar mal asignado o corresponder a una región poco específica compartida por varias proteínas.
- Cuando tenemos **varios péptidos independientes** que todos apuntan a la misma proteína, la evidencia se refuerza:
  - Es menos probable que todos sean falsos positivos.
  - También podemos cubrir diferentes regiones de la proteína y confirmar mejor su presencia.
- Además, para la **cuantificación**, usar varios péptidos permite obtener una estimación más robusta de la abundancia de la proteína (haciendo medias, descartando outliers, etc.).

En resumen, cuantos más péptidos específicos tengamos de una proteína, más sólida es la identificación y la cuantificación de esa proteína en la muestra.


---

## Ejercicio 7 — Comparación estadística entre dos grupos (versión Python)

> Tarea:  
> Evaluar si la abundancia media de los péptidos difiere significativamente entre dos grupos de muestras  
> (por ejemplo, grupo A1–A5 vs grupo A6–A10).

En el enunciado original se sugiere hacerlo en R, pero aquí lo vamos a hacer en Python con `scipy.stats`.

### 7.1. Definimos los grupos

Vamos a usar:

- Grupo 1: `A1`–`A5`
- Grupo 2: `A6`–`A10`

y trabajaremos con las columnas `LFQ intensity` correspondientes.


In [15]:
# Definimos las columnas de cada grupo para el análisis de t-test
grupo1_cols = [f"LFQ intensity A{i}" for i in range(1, 6)]  # A1-A5
grupo2_cols = [f"LFQ intensity A{i}" for i in range(6, 11)] # A6-A10

# Nos quedamos solo con las columnas que existen en el df
grupo1_cols = [c for c in grupo1_cols if c in df.columns]
grupo2_cols = [c for c in grupo2_cols if c in df.columns]

grupo1_cols, grupo2_cols

(['LFQ intensity A1',
  'LFQ intensity A2',
  'LFQ intensity A3',
  'LFQ intensity A4',
  'LFQ intensity A5'],
 ['LFQ intensity A6',
  'LFQ intensity A7',
  'LFQ intensity A8',
  'LFQ intensity A9',
  'LFQ intensity A10'])

Si el dataset no tiene exactamente estas muestras, la lista de columnas se adaptará automáticamente quedándose solo con las que existan.

### 7.2. Cálculo de medias y t-test péptido a péptido

Para cada péptido (cada fila de `df`):

1. Extraemos las intensidades del grupo 1 y grupo 2.
2. Sustituimos valores `0` por `NaN` para tratarlos como valores faltantes.
3. Opcionalmente aplicamos log2 a las intensidades (suele ayudar a que la distribución se parezca más a una normal).
4. Calculamos la media de cada grupo.
5. Aplicamos un **t-test de Student** para dos muestras independientes.
6. Guardamos el p-value resultante.


In [ ]:
def preparar_intensidades(row, cols):
    # Extraemos intensidades, cambiamos 0 por NaN
    valores = row[cols].replace(0, np.nan)

    # Forzamos a numerico (convierte strings raros en NaN)
    valores = pd.to_numeric(valores, errors="coerce")

    # Aplicamos log2 sobre floats
    return np.log2(valores.astype(float))


def t_test_por_peptido(row, cols1, cols2):
    x = preparar_intensidades(row, cols1)
    y = preparar_intensidades(row, cols2)

    # Eliminamos NaN
    x = x.dropna()
    y = y.dropna()

    # Si no hay suficientes datos, devolvemos NaN
    if len(x) < 2 or len(y) < 2:
        return np.nan, np.nan, np.nan

    # Medias
    mean_x = x.mean()
    mean_y = y.mean()

    # t-test (independiente, varianzas no iguales)
    t_stat, p_val = stats.ttest_ind(x, y, equal_var=False)

    return mean_x, mean_y, p_val


# Aplicamos la función a todo el dataframe
resultados = df.apply(
    lambda row: t_test_por_peptido(row, grupo1_cols, grupo2_cols),
    axis=1,
    result_type="expand"
)

resultados.columns = ["mean_group1", "mean_group2", "p_value"]

# Unimos estos resultados al dataframe original
df_ttest = pd.concat([df, resultados], axis=1)

df_ttest[["Sequence", "mean_group1", "mean_group2", "p_value"]].head()

C:\Users\raule\AppData\Local\Temp\ipykernel_39552\3459403519.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  valores = row[cols].replace(0, np.nan)
C:\Users\raule\AppData\Local\Temp\ipykernel_39552\3459403519.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  valores = row[cols].replace(0, np.nan)
C:\Users\raule\AppData\Local\Temp\ipykernel_39552\3459403519.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_obj

TypeError: loop of ufunc does not support argument 0 of type int which has no callable log2 method

En la tabla `df_ttest` tenemos ahora, para cada péptido:

- `mean_group1`: media log2 de intensidades en el grupo 1 (A1–A5).
- `mean_group2`: media log2 de intensidades en el grupo 2 (A6–A10).
- `p_value`: resultado de la prueba t que nos dice si la diferencia entre medias es estadísticamente significativa.

### 7.3. Seleccionamos péptidos con diferencias significativas

Usando un umbral típico de **p < 0.05**, podemos filtrar qué péptidos muestran diferencias claras entre grupos.


In [ ]:
umbral_p = 0.05
significativos = df_ttest[df_ttest["p_value"] < umbral_p].copy()

significativos[["Sequence", "mean_group1", "mean_group2", "p_value"]].head()

Si la tabla `significativos` está vacía, significa que con los datos disponibles (y el tratamiento de valores faltantes que hemos hecho) no se detectan péptidos con diferencias significativas bajo ese umbral.

En un análisis real se podrían aplicar:

- Correcciones por múltiples comparaciones (FDR de Benjamini–Hochberg).
- Umbrales adicionales sobre el tamaño del efecto (por ejemplo, |log2 fold-change| > 1).
- Visualizaciones tipo volcano plot.

Aquí nos quedamos en la parte básica, que es lo que se pide en el ejercicio.

---

## Conclusión general

En este cuaderno hemos:

- Cargado y explorado un fichero de péptidos exportado de PRIDE/MaxQuant.
- Identificado péptidos concretos y sus proteínas/genes asociados.
- Evaluado la confianza en la identificación usando el `PEP`.
- Comparado intensidades de un péptido entre muestras y detectado valores faltantes.
- Agregado información a nivel de proteína a partir de varios péptidos.
- Aplicado un t-test péptido a péptido para comparar dos grupos de muestras.

La idea principal es que, aunque el experimento de espectrometría de masas es complejo, se puede traducir a una tabla que luego analizamos con herramientas de ciencia de datos (Python/R), combinando estadísticas básicas con interpretación biológica.
